# Torrent Downloader

Paste your magnet link and run the cell.

In [ ]:
import sys,subprocess,os,time
subprocess.check_call([sys.executable,'-m','pip','install','-q','libtorrent==2.0.11'])
from google.colab import drive
drive.mount('/content/drive')
import libtorrent as lt
m=input('magnet: ').strip()
t=['udp://tracker.opentrackr.org:1337/announce','udp://open.stealth.si:80/announce','udp://tracker.torrent.eu.org:451/announce','udp://exodus.desync.com:6969/announce','udp://tracker.moeking.me:6969/announce','udp://explodie.org:6969/announce','udp://tracker1.bt.moack.co.kr:80/announce','udp://tracker.theoks.net:6969/announce']
m+=''.join(f'&tr={i}' for i in t)
d='/content/drive/MyDrive/torrents'
os.makedirs(d,exist_ok=True)
s=lt.session({'enable_dht':True,'enable_lsd':True,'enable_upnp':True,'enable_natpmp':True})
s.add_dht_router('router.bittorrent.com',6881);s.add_dht_router('router.utorrent.com',6881)
p=lt.parse_magnet_uri(m);p.save_path=d
h=s.add_torrent(p)
while not h.status().has_metadata: time.sleep(1)
ti=h.torrent_file()
for i in range(ti.files().num_files()): print(f'{i}:{ti.files().file_path(i)} {ti.files().file_size(i)/(1024**2):.2f}MB')
sel=input('indexes or all: ').strip()
if sel!='all':
    idx=[int(x) for x in sel.split(',')]
    [h.file_priority(i,0) for i in range(ti.files().num_files()) if i not in idx]
while not h.status().is_seeding:
    st=h.status()
    print(f'{st.progress*100:.1f}% {st.download_rate/1024:.0f}KB/s {st.num_peers} peers',end='')
    time.sleep(1)
print('
Done')
s.remove_torrent(h)